In [1]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [2]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

Download the source code for BATELEC I's webpage.

In [6]:
import requests
from bs4 import BeautifulSoup

# Define the URL of the webpage
url = "https://www.batelec1.com.ph/article/004/20210806045000/20210806045100"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content of the webpage with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Get the source code of the webpage
    source_code = soup.prettify()
    
    # Define the output file path
    output_file_path = "generation charge.html"
    
    # Save the source code to an HTML file
    with open(output_file_path, "w", encoding="utf-8") as file:
        file.write(source_code)
    
    print(f"Webpage saved as {output_file_path}")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Webpage saved as generation charge.html


In [59]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import time
import os
import requests

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless Chrome (no GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920x1080')

driver = webdriver.Chrome(options=options)

# URL of the page
url = "https://www.batelec1.com.ph/article/004/20210806045000/20210806045100"
driver.get(url)

# Find the dropdown element
dropdown = Select(driver.find_element(By.ID, "ratesbillmonth"))

# Create a base directory for downloads
base_download_dir = "pdf downloads"

# Check if the directory exists and create it if not
if not os.path.exists(base_download_dir):
    os.makedirs(base_download_dir)

# Base URL for the download links
base_url = "https://www.batelec1.com.ph/assets/function/downloadgenrate.php?id="

# Wait for the dropdown to be visible
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "ratesbillmonth")))

# Iterate through all options in the dropdown
for option in dropdown.options:
    billmonth = option.get_attribute('value')
    
    if billmonth:
        # Convert billmonth to the desired format (e.g., 07-2024)
        billmonth_formatted = billmonth.replace('/', '-')
        
        # Select the option
        dropdown.select_by_value(billmonth)
        time.sleep(3)  # Wait for the page to load

        # Verify that the dropdown menu shows the selected value
        selected_option = dropdown.first_selected_option.get_attribute('value')
        if selected_option != billmonth:
            print(f"Failed to select {billmonth}. Selected option is {selected_option}.")
            continue

        # Wait for the specific table or rows to be present
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//tbody/tr")))
        except Exception as e:
            print(f"Page did not load properly for billmonth {billmonth}: {e}")
            continue

        rows = driver.find_elements(By.XPATH, "//tbody/tr")

        for index, row in enumerate(rows):
            try:
                # Find the download link
                download_link = row.find_element(By.XPATH, ".//a[contains(@href, 'downloadgenrate.php')]")
                file_id = download_link.get_attribute('href').split('id=')[-1]

                # Debug print statements
                print(f"Category: {row.find_element(By.XPATH, './/td[1]').text.strip()}")
                print(f"File ID: {file_id}")

                pdf_url = base_url + file_id
                pdf_name = f"{billmonth_formatted}_{file_id}.pdf"
                pdf_path = os.path.join(base_download_dir, pdf_name)

                # Debug print statement to verify the file path
                print(f"Downloading to: {pdf_path}")

                # Download the PDF
                response = requests.get(pdf_url)
                response.raise_for_status()  # Raise an error for bad responses

                with open(pdf_path, 'wb') as file:
                    file.write(response.content)

                print(f"Downloaded: {pdf_name}")
            except Exception as e:
                print(f"Failed to find or download {pdf_url}: {e}")

# Close the driver
driver.quit()


Failed to find or download https://www.batelec1.com.ph/assets/function/downloadgenrate.php?id=20242519111937576: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//a[contains(@href, 'downloadgenrate.php')]"}
  (Session info: chrome-headless-shell=127.0.6533.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102915088 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x000000010290d764 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x000000010251c82c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000102560834 cxxbridge1$string$len + 367060
4   chromedriver                        0x0000000102556e38 cxxbridge1$string$len + 327640
5   chromedriver                        0x000000010259848c cxxbridge1$string$len + 595500
6   chromed

KeyboardInterrupt: 